In [1]:
import os 
os.chdir("../")
%pwd

'c:\\Users\\hp\\Files\\DataScience\\End-To-End Projects\\Chicken-Disease-Classification-Projects'

In [2]:
from pathlib import Path 
from dataclasses import dataclass 

@dataclass 
class PrepareCallbacksConfig:
    root_dir: Path 
    tensorboard_root_log: Path 
    checkpoint_model_filepath: Path 
    

In [4]:
from cnnClassifier.constants import * 
from cnnClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
                config_filepath= CONFIG_FILE_PATH,
                params_filepath= PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath) 
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log)
        ])

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log=Path(config.tensorboard_root_log),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callbacks_config
        

In [17]:
import tensorflow as tf
import time 
from cnnClassifier import logging

class PrepareCallbacks: 
    def __init__(self, config:PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbaks(self):
        time_stamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log, f'tb_log_at_{time_stamp}')
        logging.info(tb_running_log_dir)
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbaks,
            self._create_ckpt_callbacks
        ]

In [18]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2025-10-05 10:27:10,415] [INFO] [root:read_yaml:22] - reading yaml file -> config\config.yaml successed.
[2025-10-05 10:27:10,422] [INFO] [root:read_yaml:22] - reading yaml file -> params.yaml successed.
[2025-10-05 10:27:10,425] [INFO] [root:create_directories:42] - folder 'artifacts' already exist.
[2025-10-05 10:27:10,428] [INFO] [root:create_directories:42] - folder 'artifacts\prepare_callbacks\checkpoint' already exist.
[2025-10-05 10:27:10,430] [INFO] [root:create_directories:42] - folder 'artifacts\prepare_callbacks\tensorboard_log' already exist.
[2025-10-05 10:27:10,434] [INFO] [root:_create_tb_callbaks:13] - artifacts\prepare_callbacks\tensorboard_log\tb_log_at_2025-10-05-10-27-10
